* python 실행 경로 확인 
    - python 가상 환경을 사용 중이라면 가상 환경의 path 가 맞는지 확인한다. 


In [1]:
%gui qt5
%matplotlib inline

import matplotlib.pyplot as plt

import sys
print(sys.executable)

d:\1git\kw_condition\.venv\Scripts\python.exe


### 1. 객체 생성

In [2]:
from kw_condition import KiwoomOpenApiPlus
from kw_condition.utils import common_util

from PySide2.QtWidgets import QApplication

myApp = None
if isinstance(QApplication.instance(), type(None)):
    print('make instance')
    myApp = QApplication([])
else:
    print("use already")
    myApp = QApplication.instance()

kw_obj = KiwoomOpenApiPlus()

kw_condition.backend.KiwoomOpenApiPlus <module 'kw_condition.backend.KiwoomOpenApiPlus' from 'D:\\1git\\kw_condition\\kw_condition\\backend\\KiwoomOpenApiPlus.py'>
kw_condition.backend.KiwoomOpenApiPlus <module 'kw_condition.backend.KiwoomOpenApiPlus' from 'D:\\1git\\kw_condition\\kw_condition\\backend\\KiwoomOpenApiPlus.py'>
make instance


### 2. 서버접속
* 자동로그인이 설정되어 있는 경우 로그인이 자동으로 처리 되며, 그렇지 않은 경우 팝업 창에서 수동으로 아이디, 비밀번호 정보를 입력해야함 
- TODO
    * 자동 로그인 설정되지 않은 상태서 로그인 정보를 외부에서 입력한 경우 매크로로 자동처리 할수 있도록 해야함 pywinauto

In [3]:
import time
kw_obj.tryConnect()
common_util.process_qt_events(20)



* 14:47:44.370546 tryConnect 
* 14:47:44.933531 init_entered 
* 14:47:44.945911 tr_init_entered 
* 14:47:44.964711 disconnected_entered 
* 14:47:47.836437 _OnEventConnect 0
* 14:47:47.848992 connected_entered 
account count: 1, keyboard_boan: 1, firewall: 2
* 14:47:47.861321 tr_standby_entered 
time out!


### 3. 서버 접속 상태 확인 

In [4]:
kw_obj.isConnected()

True

### 4. TR (주식기본정보요청) - Single Data

아래 처럼 직접 TR 요청에 필요한 입력값을 설정해 요청하고, 이후 들어오는 이벤트 또한 직접 처리해주는 방식으로 사용할 수 있다.

In [7]:
rqname = '주식기본정보요청'
trcode = 'opt10001'
screen_no = '0001'  # 화면번호, 0000 을 제외한 4자리 숫자 임의로 지정, None 의 경우 내부적으로 화면번호 자동할당

inputs = {'종목코드': '005930'}

kw_obj.add_transaction(rqname, trcode, screen_no, inputs )

common_util.process_qt_events(5)

print( kw_obj.get_transaction_result(rqname) )


* 14:48:35.286740 tr_waiting_entered 
* 14:48:35.301086 request_transaction  {'rqname': '주식기본정보요청', 'trcode': 'opt10001', 'screen_no': '0001', 'inputs': {'종목코드': '005930'}}
종목코드 005930
* 14:48:39.372668 tr_standby_entered 
time out!
{'종목코드': '005930', '상한가': '+79300', '하한가': '-42700', '기준가': '61000'}


### 4 TR (주식일봉차트조회요청) - Multi Data

In [8]:
import datetime

rqname = '주식일봉차트조회요청'
trcode = 'opt10081'
screen_no = '0002'  # 화면번호, 0000 을 제외한 4자리 숫자 임의로 지정, None 의 경우 내부적으로 화면번호 자동할당

current_time_str = datetime.datetime.now().strftime('%Y%m%d')

inputs = {'종목코드': '005930', '기준일자' : current_time_str, "수정주가구분": '1'}

kw_obj.add_transaction(rqname, trcode, screen_no, inputs )

common_util.process_qt_events(5)
print( kw_obj.get_transaction_result(rqname) )

* 14:49:09.071014 tr_waiting_entered 
* 14:49:09.084908 request_transaction  {'rqname': '주식일봉차트조회요청', 'trcode': 'opt10081', 'screen_no': '0002', 'inputs': {'종목코드': '005930', '기준일자': '20220817', '수정주가구분': '1'}}
종목코드 005930
기준일자 20220817
수정주가구분 1
* 14:49:13.149202 tr_standby_entered 
time out!
